In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pickle 
import base64 
import os
from datetime import datetime
from datetime import datetime, timedelta

In [2]:
SCOPES = ["https://mail.google.com/", "https://www.googleapis.com/auth/drive.readonly", "https://www.googleapis.com/auth/gmail.modify"]#"https://www.googleapis.com/auth/gmail.send",'https://www.googleapis.com/auth/gmail.readonly','https://mail.google.com/', "https://www.googleapis.com/auth/gmail.metadata", "https://www.googleapis.com/auth/gmail.modify"]
gmail_client_secret_file = './credenciales/client_secret_python_gmail.json'

## conexion

In [3]:
def conexion_gmail_api(SCOPES,gmail_client_secret_file):
    creds = None
    # El archivo token.pickle almacena los tokens de acceso y actualización del usuario, y se
    # crea automáticamente cuando el flujo de autorización se completa por primera vez.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # Si no hay credenciales disponibles o son inválidas, pide al usuario que se loguee.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                gmail_client_secret_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # Guarda las credenciales para la próxima ejecución
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service

In [4]:
service = conexion_gmail_api(SCOPES,gmail_client_secret_file)

##### si se va a ejecutarel código en una máquina virtual de google cloud las credenciales se harán de la siguiente manera

In [5]:
"""
# Configuración de credenciales del servicio de Google Cloud
credentials = Credentials.from_service_account_file(
    'ruta/de/tu/credencial-de-servicio.json',  # Reemplaza con la ruta de tus credenciales JSON alojada en la VM
    scopes=SCOPES
)

# Obtención de las credenciales del usuario (esto abrirá una ventana del navegador para autenticación) es como está ahora
flow = InstalledAppFlow.from_client_secrets_file(
    'credentials.json',  # Reemplaza con la ruta de tu archivo de credenciales de la aplicación
    scopes=SCOPES
)
credentials = flow.run_local_server(port=0)

# Construcción del servicio de la API de Gmail
service = build('gmail', 'v1', credentials=credentials)
"""

"\n# Configuración de credenciales del servicio de Google Cloud\ncredentials = Credentials.from_service_account_file(\n    'ruta/de/tu/credencial-de-servicio.json',  # Reemplaza con la ruta de tus credenciales JSON alojada en la VM\n    scopes=SCOPES\n)\n\n# Obtención de las credenciales del usuario (esto abrirá una ventana del navegador para autenticación) es como está ahora\nflow = InstalledAppFlow.from_client_secrets_file(\n    'credentials.json',  # Reemplaza con la ruta de tu archivo de credenciales de la aplicación\n    scopes=SCOPES\n)\ncredentials = flow.run_local_server(port=0)\n\n# Construcción del servicio de la API de Gmail\nservice = build('gmail', 'v1', credentials=credentials)\n"

In [6]:
"""
# Configuración de credenciales del servicio de Google Cloud
credentials = Credentials.from_service_account_file(
    'ruta/de/tu/credencial-de-servicio.json',  # Reemplaza con la ruta de tus credenciales JSON
    scopes=SCOPES
)

# Obtención de las credenciales del usuario. Esto cambiará el flujo de autenticación y evitará la apertura de la ventana emergente en el navegador. En cambio, mostrará un enlace en la consola que el usuario puede seguir para dar permisos manualmente.
flow = InstalledAppFlow.from_client_secrets_file(
    'credentials.json',  # Reemplaza con la ruta de tu archivo de credenciales de la aplicación
    scopes=SCOPES
)

credentials = flow.run_console()

# Construcción del servicio de la API de Gmail
service = build('gmail', 'v1', credentials=credentials)
"""

"\n# Configuración de credenciales del servicio de Google Cloud\ncredentials = Credentials.from_service_account_file(\n    'ruta/de/tu/credencial-de-servicio.json',  # Reemplaza con la ruta de tus credenciales JSON\n    scopes=SCOPES\n)\n\n# Obtención de las credenciales del usuario. Esto cambiará el flujo de autenticación y evitará la apertura de la ventana emergente en el navegador. En cambio, mostrará un enlace en la consola que el usuario puede seguir para dar permisos manualmente.\nflow = InstalledAppFlow.from_client_secrets_file(\n    'credentials.json',  # Reemplaza con la ruta de tu archivo de credenciales de la aplicación\n    scopes=SCOPES\n)\n\ncredentials = flow.run_console()\n\n# Construcción del servicio de la API de Gmail\nservice = build('gmail', 'v1', credentials=credentials)\n"

## requests

### obtener campos del email

##### Queries para poner dentro del method list (como en el siguiente ejemplo de subject):
  - subject:
  - from:
  - to:
  - cc:
  - bcc:
  - label:
  - has:attachment
  - has:drive
  - has:document
  - has:spreadsheet
  - has:presentation
  - filename:
  - newer_than:2d
  - deliveredto:
  - OR or { }: --> from:amy OR from:david

https://support.google.com/mail/answer/7190?hl=en    --> All the terms and REGEXtation

### adjuntos

In [8]:
def mark_as_read(message_id):
    try:
        # Actualizar el estado del mensaje a leído
        body = {'removeLabelIds': ['UNREAD']}
        service.users().messages().modify(userId='me', id=message_id, body=body).execute()
        print(f'Email mark as READ.')

    except Exception as e:
        print(f'Error to mark the email READ: {e}')

def get_filtered_messages(subject_filter, unread_filter, sender_filter, start_date_filter, attachment_filter, attachment_file_type_filter):
    try:
        # Construcción de la consulta
        query = f'is:'

        if unread_filter:
            query += f' {unread_filter}'

        if subject_filter:
            query += f' subject:{subject_filter}'
        
        if sender_filter:
            query += f' from:{sender_filter}'

        if start_date_filter:
            formatted_date = datetime.strptime(start_date_filter, '%Y-%m-%d').strftime('%Y/%m/%d')
            query += f' after:{formatted_date}'

        if attachment_filter:
            query += f' has:{attachment_filter}'

        if attachment_file_type_filter:
            query += f' filename:{attachment_file_type_filter}'

        # Obtención de mensajes no leídos con los filtros aplicados
        response = service.users().messages().list(userId='me', q=query).execute()
        messages = response.get('messages', [])
        if messages == []:
            print('No emails in this query')

        return messages

    except Exception as e:
        print(f'Error to obtain the emails from the query: {e}')
        return []

def download_attachment(message_id, attachment_name, save_path):
    try:
        # Obtención del mensaje
        message = service.users().messages().get(userId='me', id=message_id).execute()

        # Iteración sobre las partes del mensaje
        for part in message['payload']['parts']:
            if 'filename' in part and part['filename'] == attachment_name:
                # Descarga del archivo adjunto
                attachment = service.users().messages().attachments().get(
                    userId='me', messageId=message_id, id=part['body']['attachmentId']
                ).execute()

                file_data = base64.urlsafe_b64decode(attachment['data'])

                # Guardar el archivo en el sistema local
                with open(save_path, 'wb') as f:
                    f.write(file_data)

                print(f'File download in: {save_path}')
                mark_as_read(message_id)
                return True

        print(f'Attachmen not found: {attachment_name}')
        return False

    except Exception as e:
        print(f'Error to download the file: {e}')
        return False


# Ejemplo de uso
subject_filter = 'test gmail excel'  # Reemplazar con el asunto real del correo electrónico
unread_filter = 'unread'
sender_filter = 'alvarosaezsanchez@gmail.com'  # Reemplazar con el remitente real del correo electrónico
start_date_filter = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d") #desde ayer
save_path = './files/GEO_IP_1_comb (1).xlsx'  # Reemplazar con la ruta real de destino
attachment_filter = 'attachment'
attachment_file_type_filter = 'xlsx'
attachment_name= 'GEO_IP_1_comb (1).xlsx'

filtered_messages = get_filtered_messages(subject_filter, unread_filter, sender_filter, start_date_filter,attachment_filter,attachment_file_type_filter)

for message in filtered_messages:
    message_id = message['id']
    download_attachment(message_id, attachment_name, save_path)

File download in: ./files/GEO_IP_1_comb (1).xlsx
Email mark as READ.
